In [7]:
import numpy as np
import seaborn as sns
np.random.seed(0)

* 词库里面有5个单词V：`money`, `loan`,`bank`,`river`,`stream`
* 两个话题: `T1`, `T2`
* 概率分布
 - θ1money=1/3, θ1loan=1/3, θ1bank=1/3
 - θ2bank=1/3, θ2stream=1/3, θ2river=1/3

In [63]:
vocab = ['money', 'loan', 'bank', 'river', 'stream']
z_1 = np.array([1/3, 1/3, 1/3, 0, 0])
z_2 = np.array([0, 0,1/3, 1/3, 1/3])

把两个topic分布转成phi矩阵（概率分布矩阵）

In [64]:
np.array([z_1, z_2]).T

array([[0.33333333, 0.        ],
       [0.33333333, 0.        ],
       [0.33333333, 0.33333333],
       [0.        , 0.33333333],
       [0.        , 0.33333333]])

In [9]:
phi_actual = np.array([z_1, z_2]).T.reshape(len(z_2), 2)
phi_actual

array([[0.33333333, 0.        ],
       [0.33333333, 0.        ],
       [0.33333333, 0.33333333],
       [0.        , 0.33333333],
       [0.        , 0.33333333]])

## 生成模型

In [44]:
# 生成16个文档
D =16
# 给出每个文档平均单词个数
mean_length = 10
# 根据泊松分布
# 平均数为10，大小为16个的数组
# array([10, 11,  9,  9, 18, 13,  4, 10, 10,  8, 10, 11, 16,  9, 12, 12])
len_doc = np.random.poisson(mean_length, size=D)
# 话题数
T = 2
# 从概率分布抽取单词，组成句子
docs = []
orig_topics = []
for i in range(D):
    z = np.random.randint(0, 2)
    if z == 0:
        # 是从a 中以概率P，随机选择3个, p没有指定的时候相当于是一致的分布
        words = np.random.choice(vocab, size=(len_doc[i]), p=z_1).tolist()
    else:
        words = np.random.choice(vocab, size=(len_doc[i]), p=z_2).tolist()
    orig_topics.append(z)
    docs.append(words)

# 打印生成的文本    
for doc in docs:
    print(" ".join(doc))

stream bank river bank river bank stream stream river
bank loan loan loan bank bank loan bank bank money money bank money loan loan bank
river bank bank river stream river river bank stream river river stream river river
bank money loan loan bank bank money
stream bank bank stream bank stream
stream river bank river bank bank river stream bank river
loan money money loan bank loan money money bank money bank loan
money bank money loan money bank
river river river stream river
river river bank stream river
bank money loan money bank loan loan money bank bank bank bank money
loan money loan bank loan money bank money loan loan bank
bank stream bank bank river river stream river bank river river stream stream river bank
river bank river bank bank bank stream river bank
money bank loan bank loan money
bank loan money bank loan loan bank bank loan


In [45]:
# 每个句子服从那个topic
orig_topics

[1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]

## 初始化指针

In [50]:
# 单词的位置
w_i = []
# 执行顺序
i = []
# 文章的位置
d_i = []
# 随机初始化topic分布
z_i = []
counter = 0
for doc_idx, doc in enumerate(docs):
    for words_idx, word in enumerate(doc):
        # 找到目前这个单词在V中的位置
        w_i.append(np.where(np.array(vocab) == word)[0][0])
        # 记录i
        i.append(counter)
        # 记录文章的位置
        d_i.append(doc_idx)
        # 随机初始化topic分布
        z_i.append(np.random.randint(0,T))
        counter +=1
w_i = np.array(w_i)
d_i = np.array(d_i)
z_i = np.array(z_i)

In [55]:
WT = np.zeros((len(vocab), T))
for idx, word_ in enumerate(vocab):
    topics = z_i[np.where(w_i == idx)]
    for t in range(T):
        WT[idx, t] = sum(topics==t)

DT = np.zeros((D,T))
for idx, doc in enumerate(range(D)):
    topics = z_i[np.where(d_i == idx)]
    for t in range(T):
        DT[idx, t] = sum(topics==t)

In [56]:
WT_orig = WT.copy()
DT_orig = DT.copy()

In [57]:
# 第一个单词在第一个top出现了10次，在第二个top出现了13次
WT_orig

array([[10., 13.],
       [11., 16.],
       [26., 28.],
       [20., 11.],
       [10.,  8.]])

In [58]:
# 在第一个top下出现了5，在第二个top出现了4次，类似
DT_orig

array([[5., 4.],
       [7., 9.],
       [9., 5.],
       [5., 2.],
       [3., 3.],
       [6., 4.],
       [6., 6.],
       [2., 4.],
       [4., 1.],
       [2., 3.],
       [8., 5.],
       [4., 7.],
       [8., 7.],
       [4., 5.],
       [1., 5.],
       [3., 6.]])

## Gibbs 取样

In [36]:
# 这就是我们刚刚讲的采样记录仪。记录下每一个phi的变化结果。
phi_1 = np.zeros((len(vocab),100))
phi_2 = np.zeros((len(vocab),100))

# 总共跑100次
iters = 100

# 安排一个Dirichlet先验分布（通过参数）
# 也就是课上说的 a, b 两个参数
beta = 1.
alpha = 1.

for step in range(iters):
    for current in i:
        # 把D和W分别拿出来
        doc_idx = d_i[current] 
        w_idx = w_i[current]
                
        # 并把这两个从总体集合中减去
        DT[doc_idx,z_i[current]] -= 1
        WT[w_idx,z_i[current]] -= 1
        
        # 计算新的W和D的分布
        prob_word =  (WT[w_idx,:] + beta) / (WT[:,:].sum(axis=0) + len(vocab)* beta)
        prob_document = (DT[doc_idx,:] + alpha) / (DT.sum(axis=0) + D*alpha)
        # 这其实就是对于每个topic的概率
        prob = prob_word * prob_document
        
        # 把Z更新（根据刚刚求得的prob）
        z_i[current] = np.random.choice([0,1], 1, p=prob/prob.sum())[0]

        # 更新计数器
        DT[doc_idx,z_i[current]] += 1
        WT[w_idx,z_i[current]] += 1
        
        # 记录下Phi的变化
        phi  = WT/(WT.sum(axis=0))
        phi_1[:,step] = phi[:,0]
        phi_2[:,step] = phi[:,1]

In [37]:
DT

array([[10.,  0.],
       [11.,  0.],
       [ 8.,  1.],
       [ 1.,  8.],
       [ 0., 18.],
       [ 0., 13.],
       [ 0.,  4.],
       [10.,  0.],
       [ 3.,  7.],
       [ 1.,  7.],
       [ 2.,  8.],
       [ 0., 11.],
       [ 1., 15.],
       [ 9.,  0.],
       [12.,  0.],
       [12.,  0.]])

In [38]:
WT

array([[25.,  0.],
       [27.,  0.],
       [28., 29.],
       [ 0., 28.],
       [ 0., 35.]])

In [39]:
phi  = WT/(WT.sum(axis=0))
phi

array([[0.3125    , 0.        ],
       [0.3375    , 0.        ],
       [0.35      , 0.31521739],
       [0.        , 0.30434783],
       [0.        , 0.38043478]])

In [40]:
# 统计
theta = DT/DT.sum(axis=0)
# 归一
theta = theta/np.sum(theta, axis=1).reshape(16,1) 
theta

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [0.90196078, 0.09803922],
       [0.12568306, 0.87431694],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.33014354, 0.66985646],
       [0.14110429, 0.85889571],
       [0.22330097, 0.77669903],
       [0.        , 1.        ],
       [0.07120743, 0.92879257],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [41]:
np.argmax(theta, axis=1) == orig_topics

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])